In [1]:
import Pkg; Pkg.activate(".."); Pkg.instantiate()
using Altro
using TrajectoryOptimization
const TO = TrajectoryOptimization
using RobotZoo: DoubleIntegrator
using StaticArrays, LinearAlgebra
using BenchmarkTools

 Activating environment at `~/Documents/Research/Papers/altro-mpc-icra2021/benchmarks/Project.toml`
┌ Info: Precompiling Altro [5dcf52e5-e2fb-48e0-b826-96f46d2e3e73]
└ @ Base loading.jl:1260


# Simple Rocket Landing Problem
This is a super rough approximation of the rocket landing problem, where a 3D double integrator is initialized with gravity at a high altitude with some downward velocity and has to land at the origin. The controls are constrained to lie within the 2-norm cone.

The state is a basic 3D double integrator: 
$$ \mathbf{x} = \begin{bmatrix} x \\ y \\ z \\ v_x \\ v_y \\ v_z \end{bmatrix}, \qquad \mathbf{\dot{x}} 
= \begin{bmatrix} v_x \\ v_y \\ v_z \\ u_x \\ u_y \\ u_z + g \end{bmatrix} $$

In [2]:
# Basic Setup
D = 3                                  # dimension
N = 11                                 # number of knot points
dt = 0.1                               # time step (s)
u_bnd = 500                            # control limit, norm(u) <= u_bnd

# Derived Parameters
n,m = 2D,D                             # state and control dimension
tf = (N-1)*dt                          # total time

# Initial and Final States
x0 = SA_F64[10,10,100, 0,0,-10]        # initial state
xf = @SVector zeros(n)                 # final state

# Objective weights
Q = Diagonal(@SVector fill(1.0, n))    # stage-wise quadratic state cost: (x-xf)'Q(x-xf)
R = Diagonal(@SVector fill(1e-1, m))   # stage-wise quadratic control cost: u'R*u
Qf = (N-1)*Q * 100;                    # terminal quadratic state cost

In [3]:
# Model
model = DoubleIntegrator(3, gravity=SA[0,0,-9.81])

# Constraints
cons = ConstraintList(n,m,N)
goal = GoalConstraint(xf)
bnd = NormConstraint(n, m, u_bnd, TO.SecondOrderCone(), :control)
add_constraint!(cons, bnd, 1:N-1)
add_constraint!(cons, goal, N)

# Objective
obj = LQRObjective(Q, R, Qf, xf, N)

# Problem definition
prob = Problem(model, obj, xf, tf, x0=x0, constraints=cons);

In [4]:
# Solve
solver = ALTROSolver(prob,
    show_summary = true,
    verbose = 1,
    projected_newton = false
)
solve!(solver);

iter  total  c_max       cost      info                                              
-------------------------------------------------------------------------------------
1     4       2.06940633  13856.06 
2     6       0.23748557  13860.86 
3     8       0.21599166  13864.07 
4     10      0.10825177  13879.85 
5     12      0.00988356  13892.46 
iter  total  c_max       cost      info                                              
-------------------------------------------------------------------------------------
6     14      0.00009832  13893.07 
7     16      9.8686e-08  13893.07 



SOLVE COMPLETED
 solved using the ALTRO Solver,
 part of the Altro.jl package developed by the REx Lab at Stanford and Carnegie Mellon Universities

  Solve Statistics
    Total Iterations: 16
    Solve Time: 6714.76355 (ms)

  Covergence
    Terminal Cost: 13893.067960419847
    Terminal dJ: 0.005435272414615611
    Terminal gradient: 2.2677771941930452e-5
    Terminal constraint violation: 9.868553929592849e-8
    Solve Status: SOLVE_SUCCEEDED


In [6]:
U = controls(solver)
unorm = norm.(U)
println("Maximum thrust norm: ", maximum(unorm))
println("Final state error: ", norm(xf - states(solver)[end], Inf))

Maximum thrust norm: 500.00000000003143
Final state error: 9.868553929592849e-8
